<a href="https://colab.research.google.com/github/gastonviveros2411/labo2025v/blob/main/src/ensembles/Tarea_clase4_%20420_ArbolesAzarosos_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Ensembles de Arboles de Decision

Un arbol de decisión es un modelo débil, el aumento del poder predictivo proviene al ensamblar varios arboles de decisión.
<br> Si promedio n arboles identicos, el resultados es exactamente el mismo que utilizar un solo arbol, necesito PERTURBAR cada arbol para disponer de variablidad

la variabilidad provendrá de estas fuentes:


*   Perturbar el dataset
*   Perturbar el algoritmo del arbol
*   Perturbar el dataset y el algoritmo del arbol al mismo tiempo

Se verán estos tres algoritmos


*   Arboles Azarosos
*   Random Forest
*   Gradient Boosting of Decision Trees

#### 4.01 Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



---



## 4.02 Arboles Azarosos

Arboles Azarosos es el nombre de un algoritmo trivial (por favor NO confundir con Random Forest)
Qué tipo de perturbaciones se realizan en Arboles Azarosos
* Se perturba el dataset
* No se perturba el algoritmo, es siempre rpart original

Cada  arbolito de  Arboles Azarosos se entrena sobre un dataset perturbado,  que tiene exactamente la misma cantidad de registros pero solo un subconjunto de los atributos (campos)  del dataset, tomados al azar, de los originales.
<br> En esta primera corrida, se construira cada arbol en un dataset utilizando el 50% de los campos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Nov 01 04:15:05 AM 2025"

In [2]:
# cargo las librerias necesarias
require("data.table")
require("rpart")

# Instalar rBayesianOptimization si no está instalado
if (!require("rBayesianOptimization")) {
  install.packages("rBayesianOptimization", repos="http://cran.us.r-project.org")
  library("rBayesianOptimization")
}

Loading required package: data.table

Loading required package: rpart

Loading required package: rBayesianOptimization

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rBayesianOptimization’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘iterators’, ‘lhs’, ‘foreach’, ‘GPfit’




In [4]:
# Aquí debe cargar SU semilla primigenia
PARAM <- list()
PARAM$semilla_primigenia <- 129457  # CAMBIAR POR TU SEMILLA

# feature_fraction se mantiene fijo en 0.5
PARAM$feature_fraction <- 0.5

# Cantidad de árboles fija en 32
PARAM$num_trees <- 32

# cp FIJO en -1 (no se optimiza)
PARAM$cp_fijo <- -1

# Parámetros de la Optimización Bayesiana
PARAM$bo_iteraciones <- 50  # número de iteraciones de optimización
PARAM$bo_init_points <- 10  # puntos de inicialización aleatoria

cat("Configuración de Optimización Bayesiana:\n")
cat("- Iteraciones:", PARAM$bo_iteraciones, "\n")
cat("- Puntos iniciales:", PARAM$bo_init_points, "\n")
cat("- Total de evaluaciones:", PARAM$bo_iteraciones + PARAM$bo_init_points, "\n")
cat("- Árboles por ensemble:", PARAM$num_trees, "\n")
cat("- cp FIJO:", PARAM$cp_fijo, "\n")
cat("\nTiempo estimado: 1-2 horas\n")

Configuración de Optimización Bayesiana:
- Iteraciones: 50 
- Puntos iniciales: 10 
- Total de evaluaciones: 60 
- Árboles por ensemble: 32 
- cp FIJO: -1 

Tiempo estimado: 1-2 horas


In [5]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "exp4020_bayesiano"
dir.create(experimento, showWarnings=FALSE)
setwd(paste0("/content/buckets/b1/exp/", experimento))

In [6]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [7]:
# Usamos 202107 para entrenamiento
dtrain <- dataset[foto_mes == 202107]

set.seed(PARAM$semilla_primigenia)

# Separación 70% train, 30% validation
indices_train <- sample(1:nrow(dtrain), size=0.7*nrow(dtrain))
dtrain_real <- dtrain[indices_train]
dvalidation <- dtrain[-indices_train]

# Dataset futuro (para predicción final)
dfuture <- dataset[foto_mes == 202109]
dfuture[, clase_ternaria := NA]

cat("Registros training:", nrow(dtrain_real), "\n")
cat("Registros validation:", nrow(dvalidation), "\n")
cat("Registros future:", nrow(dfuture), "\n")

Registros training: 115217 
Registros validation: 49379 
Registros future: 165093 


In [8]:
# Campos para predicción
campos_buenos <- copy(setdiff(colnames(dtrain_real), c("clase_ternaria")))
cat("Cantidad de campos disponibles:", length(campos_buenos), "\n")

Cantidad de campos disponibles: 154 


In [9]:
# Función para calcular ganancia
calcular_ganancia <- function(probs, real, corte) {
  predicho <- as.numeric(probs > corte)

  # Ganancia: BAJA+2 vale +117000, otros -3000
  ganancia <- sum(
    (predicho == 1 & real == "BAJA+2") * 117000 +
    (predicho == 1 & real != "BAJA+2") * (-3000)
  )

  return(ganancia)
}

# Función para entrenar ensemble y encontrar mejor ganancia
entrenar_ensemble <- function(minsplit, minbucket, maxdepth) {

  # cp es FIJO desde PARAM
  cp <- PARAM$cp_fijo

  # Convertir a enteros los parámetros discretos
  minsplit <- as.integer(round(minsplit))
  minbucket <- as.integer(round(minbucket))
  maxdepth <- as.integer(round(maxdepth))

  # Validación: minbucket debe ser <= minsplit/2
  if (minbucket > minsplit / 2) {
    minbucket <- as.integer(minsplit / 2)
  }

  # Validaciones adicionales
  if (minsplit < 2) minsplit <- 2
  if (minbucket < 1) minbucket <- 1
  if (maxdepth < 1) maxdepth <- 1
  if (maxdepth > 30) maxdepth <- 30

  # Vector para acumular probabilidades
  prob_acum_val <- rep(0, nrow(dvalidation))

  # Entrenar los árboles
  for (arbol in 1:PARAM$num_trees) {
    # Selección aleatoria de campos
    qty_campos <- as.integer(length(campos_buenos) * PARAM$feature_fraction)
    campos_random <- sample(campos_buenos, qty_campos)
    campos_str <- paste(campos_random, collapse=" + ")
    formulita <- paste0("clase_ternaria ~ ", campos_str)

    # Entrenar árbol con manejo de errores
    tryCatch({
      modelo <- rpart(
        formulita,
        data=dtrain_real,
        xval=0,
        control=list(
          cp=cp,
          minsplit=minsplit,
          minbucket=minbucket,
          maxdepth=maxdepth
        )
      )

      # Predicción en validación
      pred <- predict(modelo, dvalidation, type="prob")
      prob_acum_val <- prob_acum_val + pred[, "BAJA+2"]
    }, error = function(e) {
      # Si hay error, usar predicción neutra
      cat("Error en árbol", arbol, ":", e$message, "\n")
    })
  }

  # Probabilidad promedio
  prob_promedio <- prob_acum_val / PARAM$num_trees

  # Buscar mejor punto de corte (más rápido, menos granular)
  cortes <- seq(0.025, 0.5, by=0.025)  # rango más acotado
  ganancias <- sapply(cortes, function(c) {
    calcular_ganancia(prob_promedio, dvalidation$clase_ternaria, c)
  })

  mejor_ganancia <- max(ganancias)

  return(mejor_ganancia)
}

In [10]:
# Contador global para tracking
iteracion_global <- 0
mejor_ganancia_global <- -Inf
mejores_params_global <- list()

# Historial de evaluaciones
historial <- data.table(
  iteracion=integer(),
  cp=numeric(),
  minsplit=numeric(),
  minbucket=numeric(),
  maxdepth=numeric(),
  ganancia=numeric(),
  tiempo_seg=numeric()
)

# Función objetivo que optimizará la BO
# IMPORTANTE: La BO MAXIMIZA esta función
funcion_objetivo <- function(minsplit, minbucket, maxdepth) {

  iteracion_global <<- iteracion_global + 1
  tiempo_inicio <- Sys.time()

  cat(sprintf("\n[Iter %d] Evaluando: cp=%.2f (FIJO), minsplit=%d, minbucket=%d, maxdepth=%d\n",
              iteracion_global, PARAM$cp_fijo, round(minsplit), round(minbucket), round(maxdepth)))

  # Entrenar y evaluar
  ganancia <- entrenar_ensemble(minsplit, minbucket, maxdepth)

  tiempo_fin <- Sys.time()
  tiempo_transcurrido <- as.numeric(difftime(tiempo_fin, tiempo_inicio, units="secs"))

  cat(sprintf("Ganancia: $%.0f | Tiempo: %.1fs",
              ganancia, tiempo_transcurrido))

  # Actualizar mejor resultado
  if (ganancia > mejor_ganancia_global) {
    mejor_ganancia_global <<- ganancia
    mejores_params_global <<- list(
      cp=PARAM$cp_fijo,
      minsplit=round(minsplit),
      minbucket=round(minbucket),
      maxdepth=round(maxdepth)
    )
    cat(" ★ NUEVO MEJOR! ★")
  }
  cat("\n")

  # Guardar en historial
  historial <<- rbind(historial, data.table(
    iteracion=iteracion_global,
    cp=PARAM$cp_fijo,
    minsplit=round(minsplit),
    minbucket=round(minbucket),
    maxdepth=round(maxdepth),
    ganancia=ganancia,
    tiempo_seg=tiempo_transcurrido
  ))

  # Guardar archivo intermedio cada 5 iteraciones
  if (iteracion_global %% 5 == 0) {
    fwrite(historial, "historial_bayesiano_parcial.csv")
  }

  # Retornar ganancia normalizada (para ayudar a la BO)
  # La BO trabaja mejor con valores en rango razonable
  return(list(Score = ganancia / 1000000))  # normalizado a millones
}

In [11]:
# Definir los límites de búsqueda para cada hiperparámetro
# La BO puede explorar valores continuos en estos rangos

bounds <- list(
  # cp es FIJO en -1, NO se optimiza
  minsplit = c(10, 400),      # mínimo para intentar split
  minbucket = c(5, 200),      # mínimo en hoja
  maxdepth = c(3, 15)         # profundidad máxima
)

cat("Rangos de búsqueda:\n")
cat("- cp:", PARAM$cp_fijo, "(FIJO)\n")
cat("- minsplit:", bounds$minsplit, "\n")
cat("- minbucket:", bounds$minbucket, "\n")
cat("- maxdepth:", bounds$maxdepth, "\n")

Rangos de búsqueda:
- cp: -1 (FIJO)
- minsplit: 10 400 
- minbucket: 5 200 
- maxdepth: 3 15 


In [12]:
cat("\n=== INICIO DE OPTIMIZACIÓN BAYESIANA ===\n")
cat("Ensemble de", PARAM$num_trees, "árboles\n")
cat("Feature fraction:", PARAM$feature_fraction, "\n")
cat("Evaluaciones totales:", PARAM$bo_iteraciones + PARAM$bo_init_points, "\n\n")

set.seed(PARAM$semilla_primigenia)
tiempo_inicio_total <- Sys.time()

# Ejecutar optimización bayesiana
resultado_bo <- BayesianOptimization(
  FUN = funcion_objetivo,
  bounds = bounds,
  init_points = PARAM$bo_init_points,  # exploración inicial aleatoria
  n_iter = PARAM$bo_iteraciones,        # iteraciones de optimización
  acq = "ucb",                          # función de adquisición: Upper Confidence Bound
  kappa = 2.576,                        # exploración vs explotación
  eps = 0.0,
  verbose = TRUE
)

tiempo_fin_total <- Sys.time()
tiempo_total <- as.numeric(difftime(tiempo_fin_total, tiempo_inicio_total, units="mins"))

cat("\n=== OPTIMIZACIÓN COMPLETADA ===\n")
cat(sprintf("Tiempo total: %.1f minutos\n", tiempo_total))


=== INICIO DE OPTIMIZACIÓN BAYESIANA ===
Ensemble de 32 árboles
Feature fraction: 0.5 
Evaluaciones totales: 60 

elapsed = 1114.817	Round = 1	minsplit = 251.6928	minbucket = 193.7053	maxdepth = 6.08146	Value = 18.6060 
elapsed = 2256.931	Round = 2	minsplit = 366.5620	minbucket = 77.51038	maxdepth = 14.5546	Value = 19.1760 
elapsed = 921.427	Round = 3	minsplit = 122.4465	minbucket = 102.5408	maxdepth = 5.83571	Value = 17.6760 
elapsed = 2322.645	Round = 4	minsplit = 120.2613	minbucket = 14.8221	maxdepth = 13.95143	Value = 18.2790 
elapsed = 1452.552	Round = 5	minsplit = 186.6315	minbucket = 98.41465	maxdepth = 9.140456	Value = 18.7050 
elapsed = 605.113	Round = 6	minsplit = 165.7821	minbucket = 87.35007	maxdepth = 4.069812	Value = 17.5980 
elapsed = 605.04	Round = 7	minsplit = 302.8556	minbucket = 13.14099	maxdepth = 4.459303	Value = 17.4840 
elapsed = 598.609	Round = 8	minsplit = 226.5274	minbucket = 59.34101	maxdepth = 3.655112	Value = 16.9620 
elapsed = 1457.221	Round = 9	minsplit 

Timing stopped at: 1008 1.677 1022



In [ ]:
# Guardar historial completo
fwrite(historial, "historial_bayesiano_completo.csv")

# Ordenar por ganancia
historial_ordenado <- historial[order(-ganancia)]

cat("\n=== TOP 10 MEJORES CONFIGURACIONES ===\n")
print(historial_ordenado[1:min(10, nrow(historial_ordenado))])

# Mejor configuración encontrada
mejor_config <- historial_ordenado[1]

cat("\n=== MEJOR CONFIGURACIÓN ENCONTRADA ===\n")
cat(sprintf("cp: %.4f\n", mejor_config$cp))
cat(sprintf("minsplit: %d\n", mejor_config$minsplit))
cat(sprintf("minbucket: %d\n", mejor_config$minbucket))
cat(sprintf("maxdepth: %d\n", mejor_config$maxdepth))
cat(sprintf("Ganancia en validación: $%.0f\n", mejor_config$ganancia))

# Estadísticas de la búsqueda
cat("\n=== ESTADÍSTICAS DE LA BÚSQUEDA ===\n")
cat(sprintf("Ganancia mínima explorada: $%.0f\n", min(historial$ganancia)))
cat(sprintf("Ganancia máxima explorada: $%.0f\n", max(historial$ganancia)))
cat(sprintf("Ganancia promedio: $%.0f\n", mean(historial$ganancia)))
cat(sprintf("Desviación estándar: $%.0f\n", sd(historial$ganancia)))
cat(sprintf("Mejora sobre promedio: %.1f%%\n",
            100 * (mejor_config$ganancia - mean(historial$ganancia)) / mean(historial$ganancia)))

In [ ]:
# Gráfico de convergencia
historial[, mejor_hasta_ahora := cummax(ganancia)]

cat("\n=== EVOLUCIÓN DE LA BÚSQUEDA ===\n")
cat("Iteración | Ganancia Actual | Mejor Hasta Ahora\n")
cat("----------|-----------------|------------------\n")

# Mostrar cada 5 iteraciones
indices_muestra <- seq(1, nrow(historial), by=5)
for (i in indices_muestra) {
  cat(sprintf("%9d | $%14.0f | $%14.0f\n",
              historial$iteracion[i],
              historial$ganancia[i],
              historial$mejor_hasta_ahora[i]))
}

# Última iteración si no está en la muestra
if (!(nrow(historial) %in% indices_muestra)) {
  i <- nrow(historial)
  cat(sprintf("%9d | $%14.0f | $%14.0f\n",
              historial$iteracion[i],
              historial$ganancia[i],
              historial$mejor_hasta_ahora[i]))
}

In [ ]:
cat("\n=== ANÁLISIS DE SENSIBILIDAD ===\n\n")

# Dividir en cuartiles cada hiperparámetro y ver ganancia promedio
analisis_param <- function(nombre_param) {
  valores <- historial[[nombre_param]]
  cuartiles <- quantile(valores, probs=c(0, 0.25, 0.5, 0.75, 1))

  historial[, cuartil := cut(get(nombre_param), breaks=cuartiles,
                              include.lowest=TRUE, labels=1:4)]

  resumen <- historial[, .(ganancia_media = mean(ganancia),
                           ganancia_max = max(ganancia),
                           n = .N), by=cuartil]

  cat("\n", toupper(nombre_param), ":\n")
  cat(sprintf("Rango explorado: [%.2f, %.2f]\n", min(valores), max(valores)))
  print(resumen)

  historial[, cuartil := NULL]
}

analisis_param("cp")
analisis_param("minsplit")
analisis_param("minbucket")
analisis_param("maxdepth")

In [ ]:
cat("\n=== ENTRENANDO MODELO FINAL ===\n")
cat("Usando TODO el dataset de 202107 con mejor configuración\n\n")

# Preparar tabla de predicción
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob_acumulada := 0]

set.seed(PARAM$semilla_primigenia)

# Entrenar ensemble final con mejores hiperparámetros
for (arbolito in seq(PARAM$num_trees)) {
  if (arbolito %% 5 == 0) cat(arbolito, " ")

  qty_campos <- as.integer(length(campos_buenos) * PARAM$feature_fraction)
  campos_random <- sample(campos_buenos, qty_campos)
  campos_str <- paste(campos_random, collapse=" + ")
  formulita <- paste0("clase_ternaria ~ ", campos_str)

  modelo <- rpart(
    formulita,
    data=dtrain,  # TODO el dataset de 202107
    xval=0,
    control=list(
      cp=mejor_config$cp,
      minsplit=mejor_config$minsplit,
      minbucket=mejor_config$minbucket,
      maxdepth=mejor_config$maxdepth
    )
  )

  prediccion <- predict(modelo, dfuture, type="prob")
  tb_prediccion[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]
}

cat("\n\n")

# Encontrar mejor punto de corte en las probabilidades acumuladas
# Probamos diferentes umbrales
cortes_test <- seq(0.5, 2.0, by=0.05) * PARAM$num_trees
envios <- sapply(cortes_test, function(c) sum(tb_prediccion$prob_acumulada > c))

cat("Análisis de puntos de corte:\n")
cat("Umbral (prob acum) | Cantidad de envíos\n")
cat("-------------------|------------------\n")
for (i in seq_along(cortes_test)[c(1, seq(5, length(cortes_test), by=5))]) {
  cat(sprintf("%18.2f | %16d\n", cortes_test[i], envios[i]))
}

# Usar un corte que dé aproximadamente 8000-12000 envíos (ajustar según tu caso)
objetivo_envios <- 10000
idx_corte <- which.min(abs(envios - objetivo_envios))
umbral_corte <- cortes_test[idx_corte]

tb_prediccion[, Predicted := as.numeric(prob_acumulada > umbral_corte)]

archivo_kaggle <- "KA420_bayesiano_32arboles.csv"
fwrite(
  tb_prediccion[, list(numero_de_cliente, Predicted)],
  file=archivo_kaggle,
  sep=","
)

cat("\nArchivo de predicción generado:", archivo_kaggle, "\n")
cat("Umbral de corte usado:", umbral_corte, "\n")
cat("Registros predichos como BAJA+2:", sum(tb_prediccion$Predicted), "\n")

In [ ]:
# Subida a Kaggle
mensaje <- sprintf(
  "Bayesian Opt 32 arboles - cp=-1(fijo) ms=%d mb=%d md=%d",
  mejor_config$minsplit,
  mejor_config$minbucket, mejor_config$maxdepth
)

comando <- "kaggle competitions submit"
competencia <- "-c labo-i-2025-ba-analista-sr"
arch <- paste("-f", archivo_kaggle)
mensaje_cmd <- paste0("-m '", mensaje, "'")

linea <- paste(comando, competencia, arch, mensaje_cmd)
cat("\nEjecutando:", linea, "\n\n")
salida <- system(linea, intern=TRUE)
cat(salida)

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

In [ ]:
cat("\n" , "=", "=", "=", " RESUMEN FINAL ", "=", "=", "=", "\n\n")

cat("CONFIGURACIÓN ÓPTIMA ENCONTRADA:\n")
cat("--------------------------------\n")
cat(sprintf("  cp:        %.4f\n", mejor_config$cp))
cat(sprintf("  minsplit:  %d\n", mejor_config$minsplit))
cat(sprintf("  minbucket: %d\n", mejor_config$minbucket))
cat(sprintf("  maxdepth:  %d\n", mejor_config$maxdepth))
cat("\n")
cat(sprintf("Ganancia en validación: $%.0f\n", mejor_config$ganancia))
cat(sprintf("Evaluaciones realizadas: %d\n", nrow(historial)))
cat(sprintf("Tiempo total: %.1f minutos\n", tiempo_total))
cat(sprintf("Tiempo promedio por evaluación: %.1f segundos\n",
            mean(historial$tiempo_seg)))

cat("\nARCHIVOS GENERADOS:\n")
cat("-------------------\n")
cat("  • historial_bayesiano_completo.csv\n")
cat("  • KA420_bayesiano_32arboles.csv\n")

cat("\nPRÓXIMOS PASOS:\n")
cat("--------------\n")
cat("1. Comparar estos resultados con árbol único\n")
cat("2. Analizar diferencias en hiperparámetros\n")
cat("3. Verificar score en Kaggle\n")
cat("4. Documentar conclusiones\n")

cat("\n" , "=", "=", "=", "=", "=", "=", "=", "=", "=", "=", "=", "=", "\n")